## Import home made librairies

In [26]:
from optionspricer.products.option import Option
from optionspricer.products.underlying import Product
from optionspricer.pricer.blackscholes import BlackScholesPricer
from optionspricer.products.risk_free_rate import MultiCurrencyRiskFreeRate
from optionspricer.pricer.binomial import BinomialPricer
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

# Black & Scholes pricing with european option

## S&P 500 ETF based option

In [30]:
etf_sp500 = Product("SPY")
etf_data = etf_sp500.get_stock_data(historic='1y')

px.line(etf_data, x=etf_data.index, y=['Close'], title='S&P 500 based Index').show()

### Choosing an option

In [3]:
etf_sp500.options_maturities

['2025-01-06',
 '2025-01-07',
 '2025-01-08',
 '2025-01-09',
 '2025-01-10',
 '2025-01-17',
 '2025-01-24',
 '2025-01-31',
 '2025-02-07',
 '2025-02-21',
 '2025-02-28',
 '2025-03-21',
 '2025-03-31',
 '2025-04-17',
 '2025-04-30',
 '2025-05-16',
 '2025-05-30',
 '2025-06-20',
 '2025-06-30',
 '2025-07-18',
 '2025-08-15',
 '2025-09-19',
 '2025-09-30',
 '2025-12-19',
 '2025-12-31',
 '2026-01-16',
 '2026-03-20',
 '2026-06-18',
 '2026-12-18',
 '2027-01-15']

In [32]:
# Let's choose a call option that expire in next september
option_chain = etf_sp500.calls_puts_for_maturity(
    type="call", expiration_date="2025-09-19"
).sort_values(by=["volume"], ascending=False)
display(option_chain)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
80,SPY250919C00640000,2025-01-03 20:49:51+00:00,640.0,12.68,12.70,12.81,2.010000,18.837864,2343.0,8358,0.150399,False,REGULAR,USD
82,SPY250919C00650000,2025-01-03 19:30:00+00:00,650.0,9.39,9.30,9.40,1.640000,21.161295,1162.0,10852,0.143273,False,REGULAR,USD
74,SPY250919C00610000,2025-01-03 20:45:52+00:00,610.0,27.60,27.57,27.71,3.860001,16.259481,664.0,2275,0.178429,False,REGULAR,USD
99,SPY250919C00735000,2024-12-31 15:13:04+00:00,735.0,0.43,0.45,0.47,0.000000,0.000000,504.0,1387,0.123544,False,REGULAR,USD
36,SPY250919C00420000,2024-12-11 17:57:22+00:00,420.0,203.22,184.42,188.02,0.000000,0.000000,302.0,614,0.426184,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,SPY250919C00440000,2024-12-30 20:28:16+00:00,440.0,166.42,165.74,169.35,0.000000,0.000000,1.0,186,0.396765,True,REGULAR,USD
62,SPY250919C00550000,2025-01-03 19:05:19+00:00,550.0,70.82,70.10,71.56,5.389999,8.237810,1.0,1367,0.248909,True,REGULAR,USD
76,SPY250919C00620000,2025-01-03 16:36:16+00:00,620.0,21.52,21.90,22.03,2.729999,14.529001,1.0,6239,0.168244,False,REGULAR,USD
24,SPY250919C00360000,2024-12-24 18:12:42+00:00,360.0,253.33,241.23,244.85,0.000000,0.000000,1.0,20,0.521886,True,REGULAR,USD


In [5]:
# So I've choose the one with most volume.
sp500_option = Option("SPY250919C00640000")

### Option data

In [17]:
S, K, T, sigma, option_type, currency = sp500_option.get_option_data()
print(
    "Spot: {:.2f}, Strike: {}, Maturity: {:.3f}, Volatitlity: {:.4f}, Type: {}, Currency: {}".format(
        S, K, T, sigma, option_type, currency
    )
)

Spot: 591.95, Strike: 640.0, Maturity: 0.707, Volatitlity: 0.1261, Type: call, Currency: USD


Option market data

In [52]:
market_price = (
    option_chain.query("contractSymbol == 'SPY250919C00640000'")["bid"].values[0]
    + option_chain.query("contractSymbol == 'SPY250919C00640000'")["ask"].values[0]
) / 2

implied_volatility = option_chain.query("contractSymbol == 'SPY250919C00640000'")['impliedVolatility'].values[0]
print(market_price, implied_volatility)

12.754999999999999 0.15039912109375003


Option style

In [18]:
# Since Black-Scholes is applicable for European options, we need to verify the option style
style = sp500_option.infer_option_style
display(style)

'european'

### Risk free rate

In [19]:
r = MultiCurrencyRiskFreeRate().get_risk_free_rate(
    currency=sp500_option.currency,  fallback_method="proxy"
)['rate']
print("Risk free rate = {:.3%}".format(r))

Risk free rate = 4.360%


# Black & Scholes

In [53]:
# Balck Scholes price with underlying vol vs implied vol
bs_price_vol_u = BlackScholesPricer().price(
    S, K, T, sigma=sigma, r=r, option_type=option_type, style=style
)
bs_price_vol_i = BlackScholesPricer().price(
    S, K, T, sigma=implied_volatility, r=r, option_type=option_type, style=style
)
print(
    "Price with underlting vol: {:.4f}, price with implied vol: {:.4f}".format(
        bs_price_vol_u, bs_price_vol_i
    )
)

Price with underlting vol: 13.8154, price with implied vol: 18.3532


# Binomial

In [24]:
N_values = [10, 20, 50, 100, 200, 400, 800]
results = []
for N in N_values:
    bn_price = BinomialPricer.price(S, K, r, T, sigma, N=N, option_type=option_type, style = style)
    results.append((N, bn_price))

for res in results:
    print(f"N = {res[0]} => option price = {res[1]:.4f}")


N = 10 => option price = 13.5945
N = 20 => option price = 13.9493
N = 50 => option price = 13.8764
N = 100 => option price = 13.8356
N = 200 => option price = 13.8119
N = 400 => option price = 13.8216
N = 800 => option price = 13.8192
